In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import matplotlib.pyplot as plt

In [2]:
seed = 161
# random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
# os.environ['PYTHONHASHSEED'] = str(seed)

In [3]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 1e-5                  # learning rate
EPSILON = 0.5               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000

In [4]:
env = gym.make('MountainCar-v0')
# env = env.unwrapped
env.seed(seed);

In [5]:
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape

In [6]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50)
        nn.init.kaiming_uniform_(self.fc1.weight)   # initialization
        self.out = nn.Linear(50, N_ACTIONS)
        nn.init.xavier_uniform_(self.out.weight)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [7]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
#         print("Neural net")
#         print(self.eval_net)

        self.learn_step_counter = 0                                     # for target updating
        
        self.good_memory_counter = 0                                         # for storing memory
        self.good_memory = np.zeros((int(MEMORY_CAPACITY/2), N_STATES * 2 + 2))     # initialize memory
        
        self.bad_memory_counter = 0                                         # for storing memory
        self.bad_memory = np.zeros((int(MEMORY_CAPACITY/2), N_STATES * 2 + 2))     # initialize memory
        
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action
    
    def choose_greedy_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        actions_value = self.eval_net.forward(x)
        action = torch.max(actions_value, 1)[1].data.numpy()
        action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        if r > 0:
            # replace the old memory with new memory
            index = self.good_memory_counter % int(MEMORY_CAPACITY/2)
            self.good_memory[index, :] = transition
            self.good_memory_counter += 1
        
        else:
            # replace the old memory with new memory
            index = self.bad_memory_counter % int(MEMORY_CAPACITY/2)
            self.bad_memory[index, :] = transition
            self.bad_memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        good_sample_index = np.random.choice(int(MEMORY_CAPACITY/2), int(BATCH_SIZE-6))
        bad_sample_index  = np.random.choice(int(MEMORY_CAPACITY/2), int(6))

        b_good_memory = self.good_memory[good_sample_index, :]
        b_bad_memory = self.bad_memory[bad_sample_index, :]
        b_memory = np.vstack((b_good_memory,b_bad_memory))
        
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [8]:
dqn = DQN()
NO_OF_EPISODES = 1000

In [10]:
print('\nCollecting experience...')
ep_r_rec = []
time_rec = []

upgrade_metric = 200
upgrade_flag = False
upgrade_counter = 0
for i_episode in range(NO_OF_EPISODES):
    print("Episode start")
    s = env.reset()
    ep_r = 0
    time_steps = 0
    while True:
#         env.render()
        time_steps += 1
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)
    
          
        if done and s_[0] >= 0.5:
            r = 1
            print("Success")
        else:
            print("Fail")
        
        dqn.store_transition(s, a, r, s_)
        
        ep_r += r
        if dqn.bad_memory_counter > MEMORY_CAPACITY/2:
            dqn.learn()
           
        if done:
            ep_r_rec = np.append(ep_r_rec, ep_r)
            time_rec = np.append(time_rec, time_steps)
            print('EPISODE: {:4d}  |  REWARD: {:10.3f}  | TIME_STEPS: {:5d} | LR: {:3.2e} | EPSILON: {:3.2f} '.format(i_episode, ep_r, time_steps, LR, EPSILON))
            
            break
        s = s_
    
#     #if mean of episode lenght of last 10 episodes is greater than upgrade_metric
#     if np.mean(time_rec[-10:]) > upgrade_metric:
#         upgrade_counter += 1
#     else:
#         upgrade_counter = 0
        
#     if upgrade_counter > 10:
#         upgrade_counter = 0
#         print("Upgrading @ EPISODE: ", i_episode)
#         upgrade_metric *= 2
#         LR *= 0.1
#         EPSILON += 0.02
        


Episode start
Fail
1 0 -1.0
Fail
2 0 -1.0
Fail
3 0 -1.0
Fail
4 0 -1.0
Fail
5 0 -1.0
Fail
6 0 -1.0
Fail
7 0 -1.0
Fail
8 0 -1.0
Fail
9 0 -1.0
Fail
10 0 -1.0
Fail
11 0 -1.0
Fail
12 0 -1.0
Fail
13 0 -1.0
Fail
14 0 -1.0
Fail
15 0 -1.0
Fail
16 0 -1.0
Fail
17 0 -1.0
Fail
18 0 -1.0
Fail
19 0 -1.0
Fail
20 0 -1.0
Fail
21 0 -1.0
Fail
22 1 -1.0
Fail
23 1 -1.0
Fail
24 1 -1.0
Fail
25 1 -1.0
Fail
26 1 -1.0
Fail
27 1 -1.0
Fail
28 1 -1.0
Fail
29 1 -1.0
Fail
30 1 -1.0
Fail
31 1 -1.0
Fail
32 1 -1.0
Fail
33 1 -1.0
Fail
34 1 -1.0
Fail
35 1 -1.0
Fail
36 0 -1.0
Fail
37 2 -1.0
Fail
38 0 -1.0
Fail
39 0 -1.0
Fail
40 1 -1.0
Fail
41 0 -1.0
Fail
42 0 -1.0
Fail
43 0 -1.0
Fail
44 0 -1.0
Fail
45 0 -1.0
Fail
46 0 -1.0
Fail
47 0 -1.0
Fail
48 0 -1.0
Fail
49 0 -1.0
Fail
50 0 -1.0
Fail
51 0 -1.0
Fail
52 2 -1.0
Fail
53 0 -1.0
Fail
54 0 -1.0
Fail
55 0 -1.0
Fail
56 0 -1.0
Fail
57 0 -1.0
Fail
58 0 -1.0
Fail
59 0 -1.0
Fail
60 0 -1.0
Fail
61 0 -1.0
Fail
62 0 -1.0
Fail
63 0 -1.0
Fail
64 0 -1.0
Fail
65 0 -1.0
Fail
66 0 -1.0
Fail

KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize = (15,6))

ax2 = fig.add_subplot(2, 1, 1)
data = time_rec[learn_start:]
ax2.plot(data, color = 'g')
ax2.plot(np.ones_like(data)*1500, 'g--')
ax2.plot(np.ones_like(data)*200, 'r--')

ax2.set_title('Time Steps',color = 'g')
for xpt in [250,500]:
    ax2.axvline(x=xpt,color='k',linewidth='0.8')
ax2.set_yscale('log')
ax2.set_ylim([1,1e6])
fig.tight_layout()
ax2.grid()
plt.show()

In [ ]:
torch.save(dqn.eval_net.state_dict(), 'dqn_cartpole_sparse_reward_diff_mem_tough_remember_limit_2_sched.pt')

In [ ]:
env.close()

In [ ]:
# s = env.reset()
# while True:
#     env.render()
#     a = dqn.choose_greedy_action(s)
#     s_, r, done, info = env.step(a)
#     if done:
#         break
#     s = s_

In [ ]:
# env.close()